# Importing modules and functions

In [86]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import rdFingerprintGenerator
from molvs import standardize_smiles
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,KFold, StratifiedKFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import joblib
import pickle
from numpy import savetxt
from IPython.display import HTML
from rdkit.Chem import PandasTools

In [28]:
def convert_smi_to_canon_smi(smi):
    
    try:
        canon_smi = Chem.MolToSmiles(Chem.MolFromSmiles(smi),isomericSmiles = False)
    except:
        canon_smi='wrong_smiles'
    return canon_smi

In [29]:
def standart(smi):
    global m
    if smi!='wrong_smiles':
        try:
            smiles=standardize_smiles(smi)
            m = Chem.MolFromSmiles(smi)
        except:
            smi='error kekule'
    else:
        m = 'check the smiles'
    return m

# MORGAN FP_Gradient Boosting Rat oral LD50, mg/kg  

## Load data and curation work set

In [32]:
# Set file path
df_ws=pd.read_csv('datasets/rat_oral_LD50_WS.csv')
df_ws

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_oral_LD50
0,TOX-8327,12283,"6-methyl-1H-pyrimidine-2,4-dione",Cc1cc(=O)[nH]c(=O)[nH]1,Cc1cc(=O)[nH]c(=O)[nH]1,SHVCSCWHWMSGTE-UHFFFAOYSA-N,0.291207
1,TOX-27223,34042,"4-(1,3-benzodioxol-5-yl)butan-2-ylhydrazine",CC(CCc1ccc2c(c1)OCO2)NN,CC(CCc1ccc2c(c1)OCO2)NN,IBWPUTAKVGZXRB-UHFFFAOYSA-N,0.440661
2,TOX-5723,8058,hexane,CCCCCC,CCCCCC,VLKZOEOYAKHREP-UHFFFAOYSA-N,0.537456
3,TOX-72438,94201,"bis(2-ethylhexyl) cyclohexene-1,2-dicarboxylate",CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,ZVPBHZIVOWGPMT-UHFFFAOYSA-N,0.539248
4,TOX-4318,1030,"propane-1,2-diol",CC(O)CO,CC(O)CO,DNIAPMSPPWPWGF-UHFFFAOYSA-N,0.580326
...,...,...,...,...,...,...,...
7869,TOX-34548,39729,"2,3,7,8-tetrabromodibenzo-p-dioxin",Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,JZLQUWSWOJPCAK-UHFFFAOYSA-N,6.698777
7870,TOX-2774,3890,"propan-2-yl 7-[3,5-dihydroxy-2-(3-hydroxy-5-ph...",CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,GGXICVAJURFBLW-UHFFFAOYSA-N,6.937118
7871,TOX-57286,3323,"5-[2-[7a-methyl-1-[7,7,7-trifluoro-6-hydroxy-6...",C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,XPYGGHVSFMUHLH-UHFFFAOYSA-N,7.099681
7872,TOX-6661,9426,"2-[(6,7-dimethoxy-1,2,3,4-tetrahydroisoquinoli...",CCC1CN2CCc3cc(OC)c(OC)cc3C2CC1CC1NCCc2cc(OC)c(...,CCC1CN2CCc3cc(OC)c(OC)cc3C2CC1CC1NCCc2cc(OC)c(...,AUVVAXYIELKVAI-UHFFFAOYSA-N,7.602647


##  Standardization  for work set

In [34]:
df_ws["Molecule"] = df_ws.apply(lambda x: standart(x.Canonical_SMILES), axis=1)
print('Kept data: ', len(df_ws), 'molecules')

[07:44:01] Unusual charge on atom 8 number of radical electrons set to zero
[07:44:01] Unusual charge on atom 0 number of radical electrons set to zero
[07:44:01] Unusual charge on atom 16 number of radical electrons set to zero
[07:44:01] Unusual charge on atom 16 number of radical electrons set to zero


Kept data:  7874 molecules


In [35]:
df_ws

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_oral_LD50,Molecule
0,TOX-8327,12283,"6-methyl-1H-pyrimidine-2,4-dione",Cc1cc(=O)[nH]c(=O)[nH]1,Cc1cc(=O)[nH]c(=O)[nH]1,SHVCSCWHWMSGTE-UHFFFAOYSA-N,0.291207,<rdkit.Chem.rdchem.Mol object at 0x000002B4C31...
1,TOX-27223,34042,"4-(1,3-benzodioxol-5-yl)butan-2-ylhydrazine",CC(CCc1ccc2c(c1)OCO2)NN,CC(CCc1ccc2c(c1)OCO2)NN,IBWPUTAKVGZXRB-UHFFFAOYSA-N,0.440661,<rdkit.Chem.rdchem.Mol object at 0x000002B4C31...
2,TOX-5723,8058,hexane,CCCCCC,CCCCCC,VLKZOEOYAKHREP-UHFFFAOYSA-N,0.537456,<rdkit.Chem.rdchem.Mol object at 0x000002B4C31...
3,TOX-72438,94201,"bis(2-ethylhexyl) cyclohexene-1,2-dicarboxylate",CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,CCCCC(CC)COC(=O)C1=C(C(=O)OCC(CC)CCCC)CCCC1,ZVPBHZIVOWGPMT-UHFFFAOYSA-N,0.539248,<rdkit.Chem.rdchem.Mol object at 0x000002B4C31...
4,TOX-4318,1030,"propane-1,2-diol",CC(O)CO,CC(O)CO,DNIAPMSPPWPWGF-UHFFFAOYSA-N,0.580326,<rdkit.Chem.rdchem.Mol object at 0x000002B4C32...
...,...,...,...,...,...,...,...,...
7869,TOX-34548,39729,"2,3,7,8-tetrabromodibenzo-p-dioxin",Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,Brc1cc2c(cc1Br)Oc1cc(Br)c(Br)cc1O2,JZLQUWSWOJPCAK-UHFFFAOYSA-N,6.698777,<rdkit.Chem.rdchem.Mol object at 0x000002B4C33...
7870,TOX-2774,3890,"propan-2-yl 7-[3,5-dihydroxy-2-(3-hydroxy-5-ph...",CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,CC(C)OC(=O)CCCC=CCC1C(O)CC(O)C1CCC(O)CCc1ccccc1,GGXICVAJURFBLW-UHFFFAOYSA-N,6.937118,<rdkit.Chem.rdchem.Mol object at 0x000002B4C33...
7871,TOX-57286,3323,"5-[2-[7a-methyl-1-[7,7,7-trifluoro-6-hydroxy-6...",C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,C=C1C(=CC=C2CCCC3(C)C2CCC3C(C)CCCC(O)(C(F)(F)F...,XPYGGHVSFMUHLH-UHFFFAOYSA-N,7.099681,<rdkit.Chem.rdchem.Mol object at 0x000002B4C33...
7872,TOX-6661,9426,"2-[(6,7-dimethoxy-1,2,3,4-tetrahydroisoquinoli...",CCC1CN2CCc3cc(OC)c(OC)cc3C2CC1CC1NCCc2cc(OC)c(...,CCC1CN2CCc3cc(OC)c(OC)cc3C2CC1CC1NCCc2cc(OC)c(...,AUVVAXYIELKVAI-UHFFFAOYSA-N,7.602647,<rdkit.Chem.rdchem.Mol object at 0x000002B4C33...


In [36]:
y_tr=df_ws.rat_oral_LD50	
y_tr

0       0.291207
1       0.440661
2       0.537456
3       0.539248
4       0.580326
          ...   
7869    6.698777
7870    6.937118
7871    7.099681
7872    7.602647
7873    9.541127
Name: rat_oral_LD50, Length: 7874, dtype: float64

In [37]:
moldf_ws=df_ws.Molecule

##  Load data and curation test set

In [39]:
df_ts=pd.read_csv('datasets/rat_oral_LD50_TS.csv')
df_ts

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_oral_LD50
0,TOX-17137,297,methane,C,C,VNWKTOKETHGBQD-UHFFFAOYSA-N,0.017765
1,TOX-107853,17827,2-(cyclohexylamino)ethanol,OCCNC1CCCCC1,OCCNC1CCCCC1,MGUMZJAQENFQKN-UHFFFAOYSA-N,0.572835
2,TOX-44620,974,oxamic acid,NC(=O)C(=O)O,NC(=O)C(=O)O,SOWBFZRMHSNYGE-UHFFFAOYSA-N,0.624488
3,TOX-11368,62433,pentyl pentanoate,CCCCCOC(=O)CCCC,CCCCCOC(=O)CCCC,FGPPDYNPZTUNIU-UHFFFAOYSA-N,0.686956
4,TOX-7935,11429,"butane-1,2-diol",CCC(O)CO,CCC(O)CO,BMRWNKZVCUKKSR-UHFFFAOYSA-N,0.750711
...,...,...,...,...,...,...,...
1964,TOX-31209,123453,"N-(2,8,9-trioxa-5-aza-1-silabicyclo[3.3.3]unde...",c1ccc(NC[Si]23OCCN(CCO2)CCO3)cc1,c1ccc(NC[Si]23OCCN(CCO2)CCO3)cc1,YGHSPISFNKUQEY-UHFFFAOYSA-N,5.931904
1965,TOX-63960,91771,"3-[3-[4-(4-bromophenyl)phenyl]-1,2,3,4-tetrahy...",O=c1sc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,O=c1sc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,VSVAQRUUFVBBFS-UHFFFAOYSA-N,5.991624
1966,TOX-33005,38439,"1,2,3,7,8-pentachlorodibenzo-p-dioxin",Clc1cc2c(cc1Cl)Oc1c(cc(Cl)c(Cl)c1Cl)O2,Clc1cc2c(cc1Cl)Oc1c(cc(Cl)c(Cl)c1Cl)O2,FSPZPQQWDODWAU-UHFFFAOYSA-N,6.238094
1967,TOX-24685,198007,3-ethyl-N-[methoxy(methylsulfanyl)phosphoryl]-...,CCN1CC(C)OC1=NP(=O)(OC)SC,CCN1CC(C)OC1=NP(=O)(OC)SC,JCYANUVOCLKHHY-UHFFFAOYSA-N,6.401876


##  Standardization  for test set

In [41]:
df_ts["Molecule"] = df_ts.apply(lambda x: standart(x.Canonical_SMILES), axis=1)
moldf_ts=df_ts[df_ts['SMILES']!='wrong_smiles']
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  1969 molecules


In [42]:
moldf_ts

,TAID,Pubchem CID,IUPAC Name,SMILES,Canonical_SMILES,InChIKey,rat_oral_LD50,Molecule
0,TOX-17137,297,methane,C,C,VNWKTOKETHGBQD-UHFFFAOYSA-N,0.017765,<rdkit.Chem.rdchem.Mol object at 0x000002B4C41...
1,TOX-107853,17827,2-(cyclohexylamino)ethanol,OCCNC1CCCCC1,OCCNC1CCCCC1,MGUMZJAQENFQKN-UHFFFAOYSA-N,0.572835,<rdkit.Chem.rdchem.Mol object at 0x000002B4C41...
2,TOX-44620,974,oxamic acid,NC(=O)C(=O)O,NC(=O)C(=O)O,SOWBFZRMHSNYGE-UHFFFAOYSA-N,0.624488,<rdkit.Chem.rdchem.Mol object at 0x000002B4C41...
3,TOX-11368,62433,pentyl pentanoate,CCCCCOC(=O)CCCC,CCCCCOC(=O)CCCC,FGPPDYNPZTUNIU-UHFFFAOYSA-N,0.686956,<rdkit.Chem.rdchem.Mol object at 0x000002B4C41...
4,TOX-7935,11429,"butane-1,2-diol",CCC(O)CO,CCC(O)CO,BMRWNKZVCUKKSR-UHFFFAOYSA-N,0.750711,<rdkit.Chem.rdchem.Mol object at 0x000002B4C41...
...,...,...,...,...,...,...,...,...
1964,TOX-31209,123453,"N-(2,8,9-trioxa-5-aza-1-silabicyclo[3.3.3]unde...",c1ccc(NC[Si]23OCCN(CCO2)CCO3)cc1,c1ccc(NC[Si]23OCCN(CCO2)CCO3)cc1,YGHSPISFNKUQEY-UHFFFAOYSA-N,5.931904,<rdkit.Chem.rdchem.Mol object at 0x000002B4C34...
1965,TOX-63960,91771,"3-[3-[4-(4-bromophenyl)phenyl]-1,2,3,4-tetrahy...",O=c1sc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,O=c1sc2ccccc2c(O)c1C1CC(c2ccc(-c3ccc(Br)cc3)cc...,VSVAQRUUFVBBFS-UHFFFAOYSA-N,5.991624,<rdkit.Chem.rdchem.Mol object at 0x000002B4C34...
1966,TOX-33005,38439,"1,2,3,7,8-pentachlorodibenzo-p-dioxin",Clc1cc2c(cc1Cl)Oc1c(cc(Cl)c(Cl)c1Cl)O2,Clc1cc2c(cc1Cl)Oc1c(cc(Cl)c(Cl)c1Cl)O2,FSPZPQQWDODWAU-UHFFFAOYSA-N,6.238094,<rdkit.Chem.rdchem.Mol object at 0x000002B4C34...
1967,TOX-24685,198007,3-ethyl-N-[methoxy(methylsulfanyl)phosphoryl]-...,CCN1CC(C)OC1=NP(=O)(OC)SC,CCN1CC(C)OC1=NP(=O)(OC)SC,JCYANUVOCLKHHY-UHFFFAOYSA-N,6.401876,<rdkit.Chem.rdchem.Mol object at 0x000002B4C34...


In [43]:
y_ts=moldf_ts.rat_oral_LD50
y_ts

0       0.017765
1       0.572835
2       0.624488
3       0.686956
4       0.750711
          ...   
1964    5.931904
1965    5.991624
1966    6.238094
1967    6.401876
1968    7.206791
Name: rat_oral_LD50, Length: 1969, dtype: float64

In [44]:
moldf_ts=moldf_ts.Molecule

## Calculation MorganFingerprint for work set

In [88]:
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=1024)

In [91]:
fp_tr = [mfpgen.GetFingerprint(m)for m in moldf_ws]

In [94]:
def rdkit_numpy_convert(fp_tr):
    output = []
    for f in fp_tr:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [97]:
from numpy import savetxt
x_tr = rdkit_numpy_convert(fp_tr)

In [99]:
savetxt('Models/MorganFingerprint/x_tr.csv', x_tr, delimiter=',')

In [101]:
x_tr.shape

(7874, 1024)

## Calculation MorganFingerprint for test set

In [104]:
fp_ts = [mfpgen.GetFingerprint(m) for m in moldf_ts]

In [107]:
def rdkit_numpy_convert(fp_ts):
    output = []
    for f in fp_ts:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [109]:
x_ts = rdkit_numpy_convert(fp_ts)

In [111]:
x_ts.shape

(1969, 1024)

In [113]:
x_tr = np.array(x_tr, dtype=np.float32)
y_tr = np.array(y_tr, dtype=np.float32)

# CatBoostRegressor

In [122]:
cv=KFold(n_splits=5, random_state=42, shuffle=True)

In [74]:
%%time
model = CatBoostRegressor()
parameters = {'depth' : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1],
              'iterations'    : [100,500, 1000]
              }

grid = GridSearchCV(estimator=model, param_grid = parameters, n_jobs=-1, cv = cv)
grid.fit(x_tr, y_tr, verbose=False)

CPU times: total: 31min 22s
Wall time: 46min 6s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x000001D755687D00>,
             n_jobs=-1,
             param_grid={'depth': [6, 8, 10], 'iterations': [100, 500, 1000],
                         'learning_rate': [0.01, 0.05, 0.1]})

In [77]:
best_CatBR = grid.best_estimator_

In [78]:
grid.best_params_

{'depth': 10, 'iterations': 1000, 'learning_rate': 0.1}

In [80]:
y_pred_ws_GBR = best_CatBR.predict(x_tr)

In [81]:
R2_WS = round(r2_score(y_tr, y_pred_ws_GBR), 2)
R2_WS

0.92

In [82]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_GBR)), 2)
RMSE_WS

0.25

In [119]:
params={'verbose': False}

In [124]:
%%time
y_pred_CV_CatBR = cross_val_predict(best_CatBR, x_tr, y_tr, cv=cv, fit_params=params)

C:\Users\ovtti\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


CPU times: total: 1h 48min 36s
Wall time: 8min 54s


In [127]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_CatBR), 2)
Q2_CV

0.55

In [129]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_CatBR)), 2)
RMSE_CV

0.6

# save the model to disk

In [83]:
pickle.dump(best_CatBR, open('Models/MorganFingerprint/Toxicity_CatBoost_MF.pkl', 'wb'))

# load the model from disk

In [116]:
best_CatBR = pickle.load(open('Models/MorganFingerprint/Toxicity_CatBoost_MF.pkl', 'rb'))

# Prediction for test set's molecules

In [132]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [134]:
y_pred_GBR = best_CatBR.predict(x_ts)

In [136]:
Q2_TS = round(r2_score(y_ts, y_pred_GBR), 2)
Q2_TS

0.56

In [138]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_GBR)), 2)
RMSE_TS

0.59

# Estimating applicability domain. Method - Euclidian distances, K=1

In [144]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [145]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7864,7865,7866,7867,7868,7869,7870,7871,7872,7873
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.464102,4.690416,2.236068,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,3.316625,4.123106,4.690416,4.795832,2.645751,3.605551,6.557438,3.316625,5.567764,7.071068
2,4.000000,4.795832,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,3.464102,4.582576,6.164414,5.099020,5.099020,3.741657,6.557438,4.242640,6.244998,7.280110
3,4.000000,5.000000,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,4.000000,4.582576,6.244998,6.164414,5.099020,3.741657,6.633250,5.196152,6.244998,7.280110
4,4.123106,5.099020,2.449490,4.000000,2.449490,3.605551,3.741657,6.082763,2.449490,2.449490,...,4.242640,4.690416,6.244998,7.211102,6.244998,3.872983,6.633250,5.477226,6.324555,7.348469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,10.488089,10.488089,10.148891,10.392304,9.899495,10.099504,10.099504,10.440307,10.344080,10.392304,...,10.723805,10.440307,10.440307,11.532562,11.357817,10.295630,11.045361,11.357817,11.357817,11.000000
7870,10.630146,10.630146,10.583005,10.723805,10.246951,10.344080,10.677078,10.677078,10.677078,10.816654,...,11.224972,11.224972,11.224972,11.916375,11.661903,10.770329,11.090536,11.832160,11.489125,11.489125
7871,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.747340,11.661903,11.401754,12.206555,11.958261,10.954452,11.618950,12.247449,11.832160,12.083046
7872,11.269427,11.401754,11.401754,11.618950,11.180340,11.269427,11.045361,11.747340,11.575837,11.532562,...,11.832160,11.916375,11.832160,12.206555,12.124355,11.135529,11.874342,12.569805,12.083046,12.247449


In [146]:
similarity= neighbors_k

In [147]:
Dmean=np.mean(similarity[1,:])

In [148]:
round(Dmean, 2)

3.6

In [149]:
std=np.std(similarity[1,:])

In [150]:
round(std, 2)

1.22

In [151]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.21


In [152]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [153]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968
0,2.000000,3.872983,2.000000,1.414214,2.236068,2.000000,2.236068,3.605551,3.162278,3.316625,...,4.242640,7.937254,3.741657,3.000000,3.162278,4.242640,4.472136,2.645751,5.291502,2.645751
1,2.000000,4.000000,2.645751,1.732051,2.828427,2.000000,2.828427,3.741657,3.741657,3.464102,...,4.582576,8.000000,5.196152,3.162278,3.741657,4.898980,5.196152,3.741657,5.477226,3.162278
2,2.000000,4.000000,2.828427,1.732051,2.828427,2.236068,2.828427,4.000000,4.123106,3.464102,...,4.582576,8.124039,5.291502,3.605551,3.872983,4.898980,6.557438,4.000000,5.477226,3.316625
3,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.605551,...,4.582576,8.246211,5.291502,3.741657,4.000000,5.000000,6.633250,4.123106,5.567764,3.316625
4,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.741657,...,4.582576,8.246211,5.385165,3.872983,4.123106,5.000000,6.708204,4.123106,5.567764,3.464102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,9.949874,10.246951,9.899495,10.049875,10.049875,10.198039,10.099504,10.630146,10.344080,10.246951,...,10.723805,11.532562,11.000000,10.583005,10.488089,10.770329,11.269427,10.723805,10.583005,10.344080
7870,10.583005,10.583005,10.295630,10.770329,10.583005,10.630146,10.630146,10.677078,10.770329,10.770329,...,11.313708,12.041595,11.357817,10.816654,11.045361,10.816654,11.489125,10.954452,11.357817,10.677078
7871,11.135529,11.135529,10.954452,11.313708,11.135529,11.357817,11.000000,11.532562,11.401754,11.224972,...,11.747340,12.124355,11.618950,11.090536,11.135529,11.575837,12.041595,11.045361,11.532562,10.770329
7872,11.313708,11.313708,11.224972,11.313708,11.313708,11.445523,11.357817,11.532562,11.575837,11.401754,...,11.832160,12.609520,11.704700,11.269427,11.224972,11.661903,12.124355,11.224972,11.789826,10.862781


In [154]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.873 2.    ... 2.646 5.292 2.646]


In [155]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True False  True]


In [156]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.71


In [157]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1963 1966 1968]


In [158]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [176]:
y_pred_GBR_ad=list(y_pred_GBR)

In [178]:
y_pred_GBR_ad[:] = [x for i,x in enumerate(y_pred_GBR_ad) if i not in out_Ad]

In [180]:
len(y_pred_GBR_ad)

1400

In [182]:
y_ts_ad=list(y_ts)

In [184]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [186]:
len(y_ts_ad)

1400

In [188]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_GBR_ad), 2)
Q2_TS

0.64

In [190]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_GBR_ad)), 2)
RMSE_TS

0.56

# SVM model building and validation

In [112]:
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [113]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [114]:
svm = GridSearchCV(SVR(C=1.0, epsilon=0.2), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [115]:
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVR(epsilon=0.2), n_jobs=-1,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [118]:
svm.best_params_
best_svm = svm.best_estimator_

In [119]:
svm.best_params_

{'C': 10, 'gamma': 0.01}

In [120]:
y_pred_ws_svm = best_svm.predict(x_tr)

In [121]:
R2_WS = round(r2_score(y_tr, y_pred_ws_svm), 2)
R2_WS

0.86

In [122]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_svm)), 2)
RMSE_WS

0.33

In [123]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [124]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_svm), 2)
Q2_CV

0.51

In [125]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_svm)), 2)
RMSE_CV

0.63

# Prediction for test set's molecules

In [126]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [127]:
y_pred_svm = best_svm.predict(x_ts)

In [128]:
Q2_TS = round(r2_score(y_ts, y_pred_svm), 2)
Q2_TS

0.5

In [129]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_svm)), 2)
RMSE_TS

0.63

save the model to disk

In [131]:
pickle.dump(best_svm, open('models/MorganFingerprint/Toxicity_SVM_MF.pkl', 'wb'))

load the model from disk

In [53]:
best_svm = pickle.load(open('Models/Morgan_fingerprint/Toxicity_SVM_MF.pkl', 'rb'))

# Estimating applicability domain. Method - Euclidian distances, K=1

In [132]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [133]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7864,7865,7866,7867,7868,7869,7870,7871,7872,7873
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.464102,4.690416,2.236068,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,3.316625,4.123106,4.690416,4.795832,2.645751,3.605551,6.557439,3.316625,5.567764,7.071068
2,4.000000,4.795832,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,3.464102,4.582576,6.164414,5.099020,5.099020,3.741657,6.557439,4.242641,6.244998,7.280110
3,4.000000,5.000000,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,4.000000,4.582576,6.244998,6.164414,5.099020,3.741657,6.633250,5.196152,6.244998,7.280110
4,4.123106,5.099020,2.449490,4.000000,2.449490,3.605551,3.741657,6.082763,2.449490,2.449490,...,4.242641,4.690416,6.244998,7.211103,6.244998,3.872983,6.633250,5.477226,6.324555,7.348469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,10.488088,10.488088,10.148892,10.392305,9.899495,10.099505,10.099505,10.440307,10.344080,10.392305,...,10.723805,10.440307,10.440307,11.532563,11.357817,10.295630,11.045361,11.357817,11.357817,11.000000
7870,10.630146,10.630146,10.583005,10.723805,10.246951,10.344080,10.677078,10.677078,10.677078,10.816654,...,11.224972,11.224972,11.224972,11.916375,11.661904,10.770330,11.090537,11.832160,11.489125,11.489125
7871,11.180340,11.045361,11.224972,11.445523,10.908712,11.090537,10.862780,11.224972,11.313708,11.357817,...,11.747340,11.661904,11.401754,12.206556,11.958261,10.954451,11.618950,12.247449,11.832160,12.083046
7872,11.269428,11.401754,11.401754,11.618950,11.180340,11.269428,11.045361,11.747340,11.575837,11.532563,...,11.832160,11.916375,11.832160,12.206556,12.124356,11.135529,11.874342,12.569805,12.083046,12.247449


In [134]:
similarity= neighbors_k

In [135]:
Dmean=np.mean(similarity[1,:])

In [136]:
round(Dmean, 2)

3.6

In [137]:
std=np.std(similarity[1,:])

In [138]:
round(std, 2)

1.22

In [139]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.21


In [140]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [141]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968
0,2.000000,3.872983,2.000000,1.414214,2.236068,2.000000,2.236068,3.605551,3.162278,3.316625,...,4.242641,7.937254,3.741657,3.000000,3.162278,4.242641,4.472136,2.645751,5.291503,2.645751
1,2.000000,4.000000,2.645751,1.732051,2.828427,2.000000,2.828427,3.741657,3.741657,3.464102,...,4.582576,8.000000,5.196152,3.162278,3.741657,4.898979,5.196152,3.741657,5.477226,3.162278
2,2.000000,4.000000,2.828427,1.732051,2.828427,2.236068,2.828427,4.000000,4.123106,3.464102,...,4.582576,8.124038,5.291503,3.605551,3.872983,4.898979,6.557439,4.000000,5.477226,3.316625
3,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.605551,...,4.582576,8.246211,5.291503,3.741657,4.000000,5.000000,6.633250,4.123106,5.567764,3.316625
4,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.741657,...,4.582576,8.246211,5.385165,3.872983,4.123106,5.000000,6.708204,4.123106,5.567764,3.464102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,9.949874,10.246951,9.899495,10.049876,10.049876,10.198039,10.099505,10.630146,10.344080,10.246951,...,10.723805,11.532563,11.000000,10.583005,10.488088,10.770330,11.269428,10.723805,10.583005,10.344080
7870,10.583005,10.583005,10.295630,10.770330,10.583005,10.630146,10.630146,10.677078,10.770330,10.770330,...,11.313708,12.041595,11.357817,10.816654,11.045361,10.816654,11.489125,10.954451,11.357817,10.677078
7871,11.135529,11.135529,10.954451,11.313708,11.135529,11.357817,11.000000,11.532563,11.401754,11.224972,...,11.747340,12.124356,11.618950,11.090537,11.135529,11.575837,12.041595,11.045361,11.532563,10.770330
7872,11.313708,11.313708,11.224972,11.313708,11.313708,11.445523,11.357817,11.532563,11.575837,11.401754,...,11.832160,12.609520,11.704700,11.269428,11.224972,11.661904,12.124356,11.224972,11.789826,10.862780


In [142]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.873 2.    ... 2.646 5.292 2.646]


In [143]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True False  True]


In [144]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.71


In [145]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1963 1966 1968]


In [146]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [147]:
y_pred_svm_ad=list(y_pred_svm)

In [148]:
y_pred_svm_ad[:] = [x for i,x in enumerate(y_pred_svm_ad) if i not in out_Ad]

In [149]:
len(y_pred_svm_ad)

1400

In [150]:
y_ts_ad=list(y_ts)

In [151]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [152]:
len(y_ts_ad)

1400

In [153]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_svm_ad), 2)
Q2_TS

0.58

In [154]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_svm_ad)), 2)
RMSE_TS

0.6

# Multi-layer Perceptron regressor

In [155]:
from sklearn.neural_network import MLPRegressor

In [156]:
param_grid ={"hidden_layer_sizes": [(400, 300, 200, 100),(100, 100, 100), (10, 10, 10),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005], 'max_iter': [1000, 2000]}

In [157]:
m = GridSearchCV(MLPRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [158]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=MLPRegressor(), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(400, 300, 200, 100),
                                                (100, 100, 100), (10, 10, 10),
                                                (50,)],
                         'max_iter': [1000, 2000],
                         'solver': ['lbfgs', 'sgd', 'adam']},
             verbose=1)

In [159]:
best_MLPR = m.best_estimator_

In [160]:
m.best_params_

{'activation': 'relu',
 'alpha': 5e-05,
 'hidden_layer_sizes': (400, 300, 200, 100),
 'max_iter': 2000,
 'solver': 'adam'}

In [161]:
y_pred_ws_MLPR = best_MLPR.predict(x_tr)

In [162]:
R2_WS = round(r2_score(y_tr, y_pred_ws_MLPR), 2)
R2_WS

0.99

In [163]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_MLPR)), 2)
RMSE_WS

0.11

In [164]:
y_pred_CV_MLPR = cross_val_predict(best_MLPR, x_tr, y_tr, cv=cv)

In [165]:
y_pred_CV_MLPR

array([1.48264071, 2.33672459, 1.53936986, ..., 5.95051177, 2.65646771,
       2.35715558])

In [166]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_MLPR), 2)
Q2_CV

0.52

In [167]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_MLPR)), 2)
RMSE_CV

0.62

# 9. Prediction for test set's molecules

In [41]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [42]:
y_pred_MLPR = best_MLPR.predict(x_ts)

In [43]:
Q2_TS = round(r2_score(y_ts, y_pred_MLPR), 2)
Q2_TS

0.49

In [44]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_MLPR)), 2)
RMSE_TS

0.64

# save the model to disk

In [173]:
pickle.dump(best_MLPR, open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'wb'))

# load the model from disk

In [24]:
best_MLPR = pickle.load(open('models/MorganFingerprint/Toxicity_MLPR_MF.pkl', 'rb'))

#  Estimating applicability domain. Method - Euclidian distances, K=1

In [25]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [26]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7864,7865,7866,7867,7868,7869,7870,7871,7872,7873
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.464102,4.690416,2.236068,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,3.316625,4.123106,4.690416,4.795832,2.645751,3.605551,6.557438,3.316625,5.567764,7.071068
2,4.000000,4.795832,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,3.464102,4.582576,6.164414,5.099020,5.099020,3.741657,6.557438,4.242640,6.244998,7.280110
3,4.000000,5.000000,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,4.000000,4.582576,6.244998,6.164414,5.099020,3.741657,6.633250,5.196152,6.244998,7.280110
4,4.123106,5.099020,2.449490,4.000000,2.449490,3.605551,3.741657,6.082763,2.449490,2.449490,...,4.242640,4.690416,6.244998,7.211102,6.244998,3.872983,6.633250,5.477226,6.324555,7.348469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,10.488089,10.488089,10.148891,10.392304,9.899495,10.099504,10.099504,10.440307,10.344080,10.392304,...,10.723805,10.440307,10.440307,11.532562,11.357817,10.295630,11.045361,11.357817,11.357817,11.000000
7870,10.630146,10.630146,10.583005,10.723805,10.246951,10.344080,10.677078,10.677078,10.677078,10.816654,...,11.224972,11.224972,11.224972,11.916375,11.661903,10.770329,11.090536,11.832160,11.489125,11.489125
7871,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.747340,11.661903,11.401754,12.206555,11.958261,10.954452,11.618950,12.247449,11.832160,12.083046
7872,11.269427,11.401754,11.401754,11.618950,11.180340,11.269427,11.045361,11.747340,11.575837,11.532562,...,11.832160,11.916375,11.832160,12.206555,12.124355,11.135529,11.874342,12.569805,12.083046,12.247449


In [27]:
similarity= neighbors_k

In [28]:
Dmean=np.mean(similarity[1,:])

In [29]:
round(Dmean, 2)

3.6

In [30]:
std=np.std(similarity[1,:])

In [31]:
round(std, 2)

1.22

In [32]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.21


In [33]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [34]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968
0,2.000000,3.872983,2.000000,1.414214,2.236068,2.000000,2.236068,3.605551,3.162278,3.316625,...,4.242641,7.937254,3.741657,3.000000,3.162278,4.242641,4.472136,2.645751,5.291503,2.645751
1,2.000000,4.000000,2.645751,1.732051,2.828427,2.000000,2.828427,3.741657,3.741657,3.464102,...,4.582576,8.000000,5.196152,3.162278,3.741657,4.898979,5.196152,3.741657,5.477226,3.162278
2,2.000000,4.000000,2.828427,1.732051,2.828427,2.236068,2.828427,4.000000,4.123106,3.464102,...,4.582576,8.124038,5.291503,3.605551,3.872983,4.898979,6.557439,4.000000,5.477226,3.316625
3,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.605551,...,4.582576,8.246211,5.291503,3.741657,4.000000,5.000000,6.633250,4.123106,5.567764,3.316625
4,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.741657,...,4.582576,8.246211,5.385165,3.872983,4.123106,5.000000,6.708204,4.123106,5.567764,3.464102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,9.949874,10.246951,9.899495,10.049876,10.049876,10.198039,10.099505,10.630146,10.344080,10.246951,...,10.723805,11.532563,11.000000,10.583005,10.488088,10.770330,11.269428,10.723805,10.583005,10.344080
7870,10.583005,10.583005,10.295630,10.770330,10.583005,10.630146,10.630146,10.677078,10.770330,10.770330,...,11.313708,12.041595,11.357817,10.816654,11.045361,10.816654,11.489125,10.954451,11.357817,10.677078
7871,11.135529,11.135529,10.954451,11.313708,11.135529,11.357817,11.000000,11.532563,11.401754,11.224972,...,11.747340,12.124356,11.618950,11.090537,11.135529,11.575837,12.041595,11.045361,11.532563,10.770330
7872,11.313708,11.313708,11.224972,11.313708,11.313708,11.445523,11.357817,11.532563,11.575837,11.401754,...,11.832160,12.609520,11.704700,11.269428,11.224972,11.661904,12.124356,11.224972,11.789826,10.862780


In [35]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.873 2.    ... 2.646 5.292 2.646]


In [36]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True False  True]


In [37]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.71


In [38]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1963 1966 1968]


In [39]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [45]:
y_pred_MLPR_ad=list(y_pred_MLPR)

In [46]:
y_pred_MLPR_ad[:] = [x for i,x in enumerate(y_pred_MLPR_ad) if i not in out_Ad]

In [47]:
len(y_pred_MLPR_ad)

1400

In [48]:
y_ts_ad=list(y_ts)

In [49]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [50]:
len(y_ts_ad)

1400

In [51]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_MLPR_ad), 2)
Q2_TS

0.58

In [52]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_MLPR_ad)), 2)
RMSE_TS

0.6

In [53]:
from sklearn.neighbors import KNeighborsRegressor

# k-nearest neighbors

In [57]:
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

In [58]:
m = GridSearchCV(KNeighborsRegressor(), param_grid, n_jobs=-1, cv=cv, verbose=1)

In [59]:
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             verbose=1)

In [60]:
best_kNN = m.best_estimator_

In [61]:
m.best_params_

{'n_neighbors': 4}

In [62]:
y_pred_ws_kNN = best_kNN.predict(x_tr)

In [63]:
R2_WS = round(r2_score(y_tr, y_pred_ws_kNN), 2)
R2_WS

0.64

In [64]:
RMSE_WS=round(np.sqrt(mean_squared_error(y_tr, y_pred_ws_kNN)), 2)
RMSE_WS

0.54

In [65]:
y_pred_CV_kNN = cross_val_predict(best_kNN, x_tr, y_tr, cv=cv)

In [66]:
y_pred_CV_kNN

array([1.2894456, 1.9886162, 1.3313696, ..., 3.5675204, 1.875115 ,
       1.5182617], dtype=float32)

In [97]:
Q2_CV = round(r2_score(y_tr, y_pred_CV_kNN), 2)
Q2_CV

0.33

In [68]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_CV_kNN)), 2)
RMSE_CV

0.73

# 9. Prediction for test set's molecules

In [69]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [70]:
y_pred_kNN = best_kNN.predict(x_ts)

In [71]:
Q2_TS = round(r2_score(y_ts, y_pred_kNN), 2)
Q2_TS

0.34

In [72]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_kNN)), 2)
RMSE_TS

0.73

# save the model to disk

In [73]:
pickle.dump(best_kNN, open('models/MorganFingerprint/Toxicity_kNN_MF.pkl', 'wb'))

# load the model from disk

In [164]:
best_kNN = pickle.load(open('models/MorganFingerprint/Toxicity_kNN_MF.pkl', 'rb'))

#  Estimating applicability domain. Method - Euclidian distances, K=1

In [98]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [99]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,7864,7865,7866,7867,7868,7869,7870,7871,7872,7873
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.464102,4.690416,2.236068,3.741657,2.236068,3.464102,3.316625,6.000000,0.000000,2.000000,...,3.316625,4.123106,4.690416,4.795832,2.645751,3.605551,6.557438,3.316625,5.567764,7.071068
2,4.000000,4.795832,2.236068,3.872983,2.449490,3.464102,3.605551,6.082763,0.000000,2.449490,...,3.464102,4.582576,6.164414,5.099020,5.099020,3.741657,6.557438,4.242640,6.244998,7.280110
3,4.000000,5.000000,2.449490,3.872983,2.449490,3.464102,3.741657,6.082763,0.000000,2.449490,...,4.000000,4.582576,6.244998,6.164414,5.099020,3.741657,6.633250,5.196152,6.244998,7.280110
4,4.123106,5.099020,2.449490,4.000000,2.449490,3.605551,3.741657,6.082763,2.449490,2.449490,...,4.242640,4.690416,6.244998,7.211102,6.244998,3.872983,6.633250,5.477226,6.324555,7.348469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,10.488089,10.488089,10.148891,10.392304,9.899495,10.099504,10.099504,10.440307,10.344080,10.392304,...,10.723805,10.440307,10.440307,11.532562,11.357817,10.295630,11.045361,11.357817,11.357817,11.000000
7870,10.630146,10.630146,10.583005,10.723805,10.246951,10.344080,10.677078,10.677078,10.677078,10.816654,...,11.224972,11.224972,11.224972,11.916375,11.661903,10.770329,11.090536,11.832160,11.489125,11.489125
7871,11.180340,11.045361,11.224972,11.445523,10.908712,11.090536,10.862781,11.224972,11.313708,11.357817,...,11.747340,11.661903,11.401754,12.206555,11.958261,10.954452,11.618950,12.247449,11.832160,12.083046
7872,11.269427,11.401754,11.401754,11.618950,11.180340,11.269427,11.045361,11.747340,11.575837,11.532562,...,11.832160,11.916375,11.832160,12.206555,12.124355,11.135529,11.874342,12.569805,12.083046,12.247449


In [100]:
similarity= neighbors_k

In [101]:
Dmean=np.mean(similarity[1,:])

In [102]:
round(Dmean, 2)

3.6

In [103]:
std=np.std(similarity[1,:])

In [104]:
round(std, 2)

1.22

In [105]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

4.21


In [106]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [107]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968
0,2.000000,3.872983,2.000000,1.414214,2.236068,2.000000,2.236068,3.605551,3.162278,3.316625,...,4.242640,7.937254,3.741657,3.000000,3.162278,4.242640,4.472136,2.645751,5.291502,2.645751
1,2.000000,4.000000,2.645751,1.732051,2.828427,2.000000,2.828427,3.741657,3.741657,3.464102,...,4.582576,8.000000,5.196152,3.162278,3.741657,4.898980,5.196152,3.741657,5.477226,3.162278
2,2.000000,4.000000,2.828427,1.732051,2.828427,2.236068,2.828427,4.000000,4.123106,3.464102,...,4.582576,8.124039,5.291502,3.605551,3.872983,4.898980,6.557438,4.000000,5.477226,3.316625
3,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.605551,...,4.582576,8.246211,5.291502,3.741657,4.000000,5.000000,6.633250,4.123106,5.567764,3.316625
4,2.000000,4.123106,3.000000,2.000000,3.000000,2.236068,3.162278,4.000000,4.123106,3.741657,...,4.582576,8.246211,5.385165,3.872983,4.123106,5.000000,6.708204,4.123106,5.567764,3.464102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7869,9.949874,10.246951,9.899495,10.049875,10.049875,10.198039,10.099504,10.630146,10.344080,10.246951,...,10.723805,11.532562,11.000000,10.583005,10.488089,10.770329,11.269427,10.723805,10.583005,10.344080
7870,10.583005,10.583005,10.295630,10.770329,10.583005,10.630146,10.630146,10.677078,10.770329,10.770329,...,11.313708,12.041595,11.357817,10.816654,11.045361,10.816654,11.489125,10.954452,11.357817,10.677078
7871,11.135529,11.135529,10.954452,11.313708,11.135529,11.357817,11.000000,11.532562,11.401754,11.224972,...,11.747340,12.124355,11.618950,11.090536,11.135529,11.575837,12.041595,11.045361,11.532562,10.770329
7872,11.313708,11.313708,11.224972,11.313708,11.313708,11.445523,11.357817,11.532562,11.575837,11.401754,...,11.832160,12.609520,11.704700,11.269427,11.224972,11.661903,12.124355,11.224972,11.789826,10.862781


In [108]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[2.    3.873 2.    ... 2.646 5.292 2.646]


In [109]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True  True  True ...  True False  True]


In [110]:
print("Coverage = ", round(sum(cpd_AD) / len(cpd_AD), 2))

Coverage =  0.71


In [111]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [   0    1    2 ... 1963 1966 1968]


In [112]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# 12. Prediction only for molecules included in  AD

In [113]:
y_pred_kNN_ad=list(y_pred_kNN)

In [114]:
y_pred_kNN_ad[:] = [x for i,x in enumerate(y_pred_kNN_ad) if i not in out_Ad]

In [115]:
len(y_pred_kNN_ad)

1400

In [116]:
y_ts_ad=list(y_ts)

In [117]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [118]:
len(y_ts_ad)

1400

In [119]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_kNN_ad), 2)
Q2_TS

0.46

In [120]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_kNN_ad)), 2)
RMSE_TS

0.68